In [1]:
import numpy as np
import cv2
import mediapipe as mp
import pandas as pd
import os

In [2]:
mp_face_mes = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mes.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [3]:
images_sad = os.listdir('face_sad')
images_happy = os.listdir('face_happy')
images_neutral = os.listdir('face_neutral')
images_angry = os.listdir('face_angry')
images_surprised = os.listdir('face_surprised')
images_fearful = os.listdir('face_fearful')
images_disgusted = os.listdir('face_disgusted')

print(len(images_sad), len(images_happy), len(images_neutral), len(images_angry), len(images_surprised), len(images_fearful), len(images_disgusted))

6078 8990 6199 4954 4003 5122 684


In [4]:
def inference_facemesh(image):
    try:
        frame = image.copy()
    except AttributeError:
        return None
    H, W, _ = frame.shape
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_mesh = face_mesh.process(rgb_image)
    if results_mesh.multi_face_landmarks:
        mesh_points=np.array([np.multiply([p.x, p.y], [W, H]).astype(int) for p in results_mesh.multi_face_landmarks[0].landmark])
        return mesh_points
    return None

In [5]:
meshes_surprised = []
for image in images_surprised:
    img = cv2.imread('face_surprised/'+image)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_surprised.append(mesh)
print("meshes surprised: ", len(meshes_surprised))

meshes_disgusted = []
for image in images_disgusted:
    img = cv2.imread('face_disgusted/'+image)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_disgusted.append(mesh)
print("meshes disgusted: ", len(meshes_disgusted))

meshes_fearful = []
for image in images_fearful:
    img = cv2.imread('face_fearful/'+image)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_fearful.append(mesh)
print("meshes fearful: ", len(meshes_fearful))

meshes_sad = []
for image in images_sad:
    img = cv2.imread('face_sad/'+image)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_sad.append(mesh)
print("meshes sad: ", len(meshes_sad))

meshes_happy = []
for image in images_happy:
    img = cv2.imread('face_happy/'+image)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_happy.append(mesh)
print("meshes happy: ", len(meshes_happy))

meshes_neutral = []
for image in images_neutral:
    img = cv2.imread('face_neutral/'+image)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_neutral.append(mesh)
print("meshes neutral: ", len(meshes_neutral))

meshes_angry = []
for image in images_angry:
    img = cv2.imread('face_angry/'+image)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_angry.append(mesh)
print("meshes angry: ", len(meshes_angry))

meshes surprised:  3745
meshes disgusted:  578
meshes fearful:  4268
meshes sad:  4673
meshes happy:  8085
meshes neutral:  5564
meshes angry:  4204


In [6]:
def normalize_landmarks(mesh):
    nose_tip = mesh[4] 
    forehead = mesh[151]
    mesh_norm = mesh - nose_tip
    scale_factor = np.linalg.norm(forehead - nose_tip)
    if np.isclose(scale_factor, 0):
        scale_factor = 1e-6
    mesh_norm = np.divide(mesh_norm, scale_factor)
    landmarks_flat = mesh_norm.flatten()
    return landmarks_flat

In [7]:
meshes_sad_norm = []
for mesh in meshes_sad:
    mesh_norm = normalize_landmarks(mesh)
    meshes_sad_norm.append(mesh_norm)

meshes_happy_norm = []
for mesh in meshes_happy:
    mesh_norm = normalize_landmarks(mesh)
    meshes_happy_norm.append(mesh_norm)

meshes_neutral_norm = []
for mesh in meshes_neutral:
    mesh_norm = normalize_landmarks(mesh)
    meshes_neutral_norm.append(mesh_norm)

meshes_angry_norm = []
for mesh in meshes_angry:
    mesh_norm = normalize_landmarks(mesh)
    meshes_angry_norm.append(mesh_norm)

meshes_surprised_norm = []
for mesh in meshes_surprised:
    mesh_norm = normalize_landmarks(mesh)
    meshes_surprised_norm.append(mesh_norm)

meshes_fearful_norm = []
for mesh in meshes_fearful:
    mesh_norm = normalize_landmarks(mesh)
    meshes_fearful_norm.append(mesh_norm)

meshes_disgusted_norm = []
for mesh in meshes_disgusted:
    mesh_norm = normalize_landmarks(mesh)
    meshes_disgusted_norm.append(mesh_norm)

print(len(meshes_sad_norm), len(meshes_happy_norm), len(meshes_neutral_norm), len(meshes_angry_norm), len(meshes_surprised_norm), len(meshes_fearful_norm), len(meshes_disgusted_norm))

4673 8085 5564 4204 3745 4268 578


In [8]:
df_sad = pd.DataFrame(meshes_sad_norm)
df_happy = pd.DataFrame(meshes_happy_norm)
df_neutral = pd.DataFrame(meshes_neutral_norm)
df_angry = pd.DataFrame(meshes_angry_norm)
df_surprise = pd.DataFrame(meshes_surprised_norm)
df_fear = pd.DataFrame(meshes_fearful_norm)
df_disgust = pd.DataFrame(meshes_disgusted_norm)

In [9]:
df_happy.describe()

,0,1,2,3,4,5,6,7,8,9,...,946,947,948,949,950,951,952,953,954,955
count,8085.000000,8085.000000,8085.000000,8085.000000,8085.000000,8085.000000,8085.000000,8085.000000,8085.0,8085.0,...,8085.000000,8085.000000,8085.000000,8085.000000,8085.000000,8085.000000,8085.000000,8085.000000,8085.000000,8085.000000
mean,0.004781,0.330147,0.002036,0.087884,0.001652,0.164484,-0.076714,-0.201261,0.0,0.0,...,0.438479,-0.494717,0.523061,-0.495320,0.436802,-0.571251,0.352198,-0.494947,0.438960,-0.419462
std,0.130506,0.098759,0.026139,0.026640,0.067343,0.056077,0.054867,0.030358,0.0,0.0,...,0.186789,0.115744,0.195446,0.124677,0.192798,0.113724,0.180527,0.107835,0.182116,0.118329
min,-0.687118,0.041595,-0.117851,0.000000,-0.306786,0.000000,-0.320092,-0.292603,0.0,0.0,...,-0.432512,-0.988372,-0.384455,-1.064400,-0.480569,-1.064400,-0.480569,-0.988372,-0.384455,-0.912343
25%,-0.061898,0.262794,0.000000,0.058824,-0.047619,0.133038,-0.108465,-0.221880,0.0,0.0,...,0.333333,-0.570782,0.415945,-0.571863,0.332820,-0.642809,0.257513,-0.565685,0.344904,-0.497947
50%,0.000000,0.316228,0.000000,0.094809,0.000000,0.155963,-0.076696,-0.203069,0.0,0.0,...,0.444444,-0.500000,0.525786,-0.500000,0.443760,-0.578947,0.360000,-0.500000,0.447767,-0.428571
75%,0.086226,0.386510,0.000000,0.105263,0.049447,0.191663,-0.049752,-0.186052,0.0,0.0,...,0.549314,-0.436564,0.642529,-0.428571,0.550000,-0.514558,0.462573,-0.437500,0.549314,-0.355995
max,0.598671,0.971286,0.131306,0.216930,0.304114,0.582772,0.191663,0.065372,0.0,0.0,...,1.377997,0.256307,1.483997,0.383326,1.377997,0.255551,1.271997,0.191663,1.377997,0.353553


In [10]:
df_sad.describe()

,0,1,2,3,4,5,6,7,8,9,...,946,947,948,949,950,951,952,953,954,955
count,4673.000000,4673.000000,4673.000000,4673.000000,4673.000000,4673.000000,4673.000000,4673.000000,4673.0,4673.0,...,4673.000000,4673.000000,4673.000000,4673.000000,4673.000000,4673.000000,4673.000000,4673.000000,4673.000000,4673.000000
mean,0.011026,0.361860,0.002338,0.083391,0.006033,0.156486,-0.072563,-0.202403,0.0,0.0,...,0.434995,-0.514605,0.516843,-0.515376,0.433944,-0.589536,0.353157,-0.514505,0.435771,-0.439676
std,0.156487,0.107642,0.028149,0.027018,0.087254,0.058697,0.057835,0.030317,0.0,0.0,...,0.202337,0.120714,0.212378,0.130985,0.207498,0.119484,0.193569,0.111700,0.198649,0.122883
min,-0.624038,-0.140720,-0.110432,0.000000,-0.311891,-0.087706,-0.300000,-0.333333,0.0,0.0,...,-0.350823,-1.024295,-0.350823,-1.152332,-0.428746,-1.152332,-0.395663,-0.955779,-0.350823,-0.955779
25%,-0.076696,0.287698,0.000000,0.055470,-0.047891,0.117647,-0.105118,-0.221880,0.0,0.0,...,0.317854,-0.590300,0.398015,-0.596559,0.315353,-0.665912,0.245145,-0.587220,0.326860,-0.521247
50%,0.000000,0.362143,0.000000,0.090075,0.000000,0.149813,-0.071247,-0.206010,0.0,0.0,...,0.443760,-0.521450,0.523810,-0.521450,0.443242,-0.593362,0.363261,-0.521356,0.444444,-0.444478
75%,0.095238,0.428571,0.000000,0.103975,0.052559,0.190476,-0.047405,-0.186052,0.0,0.0,...,0.555556,-0.444444,0.645942,-0.441726,0.555889,-0.523424,0.471188,-0.447767,0.555556,-0.367912
max,0.640184,0.995893,0.143592,0.200000,0.512148,0.452679,0.159617,0.000000,0.0,0.0,...,1.165998,0.393073,1.271997,0.458585,1.218998,0.327561,1.059998,0.327561,1.165998,0.458585


In [11]:
df_sad.to_csv('df_sad.csv', index=False)
df_happy.to_csv('df_happy.csv', index=False)
df_neutral.to_csv('df_neutral.csv', index=False)
df_angry.to_csv('df_angry.csv', index=False)
df_surprise.to_csv('df_surprise.csv', index=False)
df_disgust.to_csv('df_disgust.csv', index=False)
df_fear.to_csv('df_fear.csv', index=False)